In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import os
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.stat import Correlation
#import seaborn as sns
import matplotlib.pyplot as plt
import pyspark.sql.functions as F
#import holidays
from datetime import datetime, timezone
from pyspark.ml.classification import RandomForestClassifier, BinaryLogisticRegressionSummary
from pyspark.ml import Pipeline
from pyspark.mllib.evaluation import MulticlassMetrics, BinaryClassificationMetrics
from pyspark.ml.pipeline import PipelineModel

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("US_Accidents") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.memory.offHeap.enabled", "true") \
    .config("spark.memory.offHeap.size", "2g") \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR") #supress warnings

In [ ]:
# Read in Data
df = spark.read.parquet("final_dataset_revised.parquet")

In [ ]:
# Use StringIndexer for encoding the 'Severity' column
indexer = StringIndexer(inputCol="Severity", outputCol="SeverityIndex")
df = indexer.fit(df).transform(df)

In [ ]:
# Create list of features
feature_list = []
for col in df.columns:
    if col == 'Severity':
        continue
    elif col == 'SeverityIndex':
        continue
    else:
        feature_list.append(col)

In [ ]:
# Split the data into train and test
splits = df.randomSplit([0.8, 0.2], 314)
train = splits[0]
test = splits[1]

In [ ]:
# Undersampling
from pyspark.sql import functions as F

# Step 1: Group by 'Severity' and count occurrences
class_counts = train.groupBy("SeverityIndex").count()

# Step 2: Use PySpark's min() function to find the minimum count
min_class_size = class_counts.agg(F.min('count')).collect()[0][0]

undersampled_train_list = []

for row in class_counts.collect():
    class_label = row['SeverityIndex']
    class_size = row['count']

    if class_size > min_class_size:
        # Sample the data for this class to the size of the minimum class
        class_data = train.filter(F.col("SeverityIndex") == class_label)
        class_data_undersampled = class_data.sample(withReplacement=False, fraction=min_class_size / class_size)
    else:
        # For classes that are already at the minimum size, keep all samples
        class_data_undersampled = train.filter(F.col("SeverityIndex") == class_label)

    undersampled_train_list.append(class_data_undersampled)

# Combine all the undersampled DataFrames
undersampled_train = undersampled_train_list[0]  # start with the first one
for df in undersampled_train_list[1:]:
    undersampled_train = undersampled_train.union(df)

# Show the result
undersampled_train.show()

# Step 4: Group by 'Severity' and count the occurrences in the undersampled DataFrame
undersampled_class_counts = undersampled_train.groupBy("SeverityIndex").count()

# Show the result
undersampled_class_counts.show()

In [ ]:
# Assemble data for logistic regression model
assembler = VectorAssembler(inputCols=feature_list,
                            outputCol="features")

undersampled_train = assembler.transform(undersampled_train)
test = assembler.transform(test)

In [ ]:
# Standardize the predictors
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
scalerModel = scaler.fit(undersampled_train)
scaledTrainData = scalerModel.transform(undersampled_train)
scaledTestData = scalerModel.transform(test)

In [ ]:
# Fit logistic regression model with intercept
from pyspark.ml.classification import LogisticRegression

# instantiate the model
lr = LogisticRegression(labelCol='SeverityIndex',
                        featuresCol='scaledFeatures',
                        #maxIter=10, 
                        #regParam=0.3, 
                        #elasticNetParam=0.8,
                        family="multinomial")

# Fit the model
lrModel = lr.fit(scaledTrainData)
print(f"Detected number of classes: {lrModel.numClasses}")

# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModel.coefficientMatrix))
print("Intercept: " + str(lrModel.interceptVector))

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# compute predictions. this will append column "prediction" to dataframe
lrPred = lrModel.transform(scaledTestData)
lrPred.select("prediction").distinct().show()

evaluator = MulticlassClassificationEvaluator(labelCol='SeverityIndex', predictionCol='prediction', metricName='accuracy')
accuracy = evaluator.evaluate(lrPred, {evaluator.metricName: "accuracy"})
print(f'Accuracy: {accuracy}')

In [ ]:
precision = evaluator.evaluate(lrPred, {evaluator.metricName: 'weightedPrecision'})
recall = evaluator.evaluate(lrPred, {evaluator.metricName: 'weightedRecall'})
f1_score = evaluator.evaluate(lrPred, {evaluator.metricName: 'f1'})

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1_score}')

In [ ]:
# Add a column to indicate correct or incorrect predictions
predictions = lrPred.withColumn(
    'is_correct', F.expr("CASE WHEN SeverityIndex = prediction THEN 1 ELSE 0 END")
)

# Calculate accuracy by class
accuracy_by_class = predictions.groupBy('SeverityIndex').agg(
    (F.sum('is_correct') / F.count('SeverityIndex')).alias('accuracy')
)

# Show per-class accuracy
accuracy_by_class.show()